In [1]:
from mysql.connector import MySQLConnection, Error
from mydbutils import make_connection, do_query_return_all
from pandas.io import sql
import pandas as pd
import csv
import random
random.seed(1)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'


import warnings
warnings.filterwarnings('ignore')

# 1 Dimension Table Customer

## 1.1 Extract and transform data

In [2]:
## Connect to 'daydayup_db' to extract data
conn = make_connection(config_file = 'config_sjsu_daydayup_db2.ini')
cursor = conn.cursor()
conn

In [3]:
## Extract data from the relational schema on database 'daydayup_db'

df_customer = pd.read_sql("SELECT * from Customer", conn)

## Drop the columns "Fax" and "Company"
## df_customer.drop(columns=['Fax', 'Company'], axis=1, inplace=True)
df_customer

,CustomerId,FirstName,LastName,Address,City,State,Country,PostalCode,Phone,Email,SupportRepId
0,1,Luís,Gonçalves,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,luisg@embraer.com.br,3
1,2,Leonie,Köhler,Theodor-Heuss-Straße 34,Stuttgart,Unknown,Germany,70174,+49 0711 2842222,leonekohler@surfeu.de,5
2,3,François,Tremblay,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,Ullevålsveien 14,Oslo,Unknown,Norway,0171,+47 22 44 22 22,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,Klanova 9/506,Prague,Unknown,Czech Republic,14700,+420 2 4172 5555,frantisekw@jetbrains.com,4
5,6,Helena,Holý,Rilská 3174/6,Prague,Unknown,Czech Republic,14300,+420 2 4177 0449,hholy@gmail.com,5
6,7,Astrid,Gruber,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,Unknown,Austria,1010,+43 01 5134505,astrid.gruber@apple.at,5
7,8,Daan,Peeters,Grétrystraat 63,Brussels,Unknown,Belgium,1000,+32 02 219 03 03,daan_peeters@apple.be,4
8,9,Kara,Nielsen,Sønder Boulevard 51,Copenhagen,Unknown,Denmark,1720,+453 3331 9991,kara.nielsen@jubii.dk,4
9,10,Eduardo,Martins,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,eduardo@woodstock.com.br,4


In [4]:
## Rplace all the "None" values with "Unknown", use the "Unknow" as a group later in data analysis
df_customer.fillna(value="Unknown", inplace=True)
df_customer.head()
cursor.close()
conn.close()

## 1.2 Create and populate the dimension table Customer (wh2)

In [5]:
## Transform the data: 1) add a primary key for dimension table, 2) add additional data for analysis
df_customer_wh2 = df_customer
df_customer_wh2['CustomerKey'] = [i for i in range(1, df_customer.shape[0]+1)]

In [6]:
## Suppose that we get the gender and age infomation
df_customer_wh2['Gender'] = ['F', 'M', 'F', 'F', 'M', 'M', 'F', 'M', 'F', 'M',
                'M', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'F',
                'F', 'F', 'F', 'F', 'M', 'F', 'F', 'F', 'F', 'M',
                'M', 'M', 'M', 'F', 'M', 'M', 'F', 'M', 'M', 'M',
                'F', 'F', 'F', 'M', 'F', 'F', 'M', 'M', 'F', 'F',
                'M', 'M', 'M', 'F', 'M', 'M', 'F', 'M', 'M']


#df_customer_wh2['Age'] = [np.random.randint(15, 100) for _ in range(df_customer.shape[0])]
#np.random.seed(1)

df_customer_wh2['Age'] = [18, 35, 30, 55, 32,  25, 33, 36, 25, 40,
                          34, 64, 19, 21, 34,  55, 71, 81, 90, 23, 
                          33, 35, 36, 38, 42, 51, 37, 23, 39, 42,
                          31, 25, 36, 38, 42, 51, 32, 28, 31, 40,
                          30, 35, 26, 38, 42, 51, 37, 29, 39, 42,
                          20, 25, 36, 38, 42, 51, 34, 28, 35]

In [7]:
df_customer.columns

Index(['CustomerId', 'FirstName', 'LastName', 'Address', 'City', 'State',
       'Country', 'PostalCode', 'Phone', 'Email', 'SupportRepId',
       'CustomerKey', 'Gender', 'Age'],
      dtype='object')

In [8]:
## Select needed columns
df_customer_wh2 = df_customer[['CustomerKey', 'CustomerId', 'FirstName', 'LastName', 'Gender', 'Age', 'Phone', 'Email', 'SupportRepId']]

In [9]:
df_customer_wh2.head(1)

,CustomerKey,CustomerId,FirstName,LastName,Gender,Age,Phone,Email,SupportRepId
0,1,1,Luís,Gonçalves,F,18,+55 (12) 3923-5555,luisg@embraer.com.br,3


In [10]:
## Export the transformed data to csv
df_customer_wh2.to_csv('wh2_customer.csv', index = False)

In [11]:
## Connect to 'daydayup_wh2' to create dimension table
conn = make_connection(config_file = 'config_sjsu_daydayup_wh2.ini')
cursor = conn.cursor()

In [12]:
## Drop table if already exist
cursor.execute('DROP TABLE IF EXISTS Sales_Fact_Table')
cursor.execute('DROP TABLE IF EXISTS Preference_Fact_Table')
cursor.execute('DROP TABLE IF EXISTS Customer')

In [13]:
## Create the dimension table Customer
sql = ('''
        CREATE TABLE Customer
        (CustomerKey INT NOT NULL AUTO_INCREMENT,
        `CustomerId` INT NOT NULL,
        `FirstName` NVARCHAR(40) NOT NULL,
        `LastName` NVARCHAR(20) NOT NULL,
        `Gender` NVARCHAR(10) NOT NULL,
        `Age` INT NOT NULL,
        `Phone` NVARCHAR(30),
        `Email` NVARCHAR(60) NOT NULL,
        `SupportRepId` INT,
         PRIMARY KEY (CustomerKey))
        ''')

cursor.execute(sql)

In [14]:
## Populate the dimension table CUSTOMER
sql_c = ( "INSERT INTO Customer \n"
        + "VALUES (%s, %s, %s, %s, %s,   %s, %s, %s, %s)"
        )


first = True ## Skip the first row

with open('wh2_customer.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            #transform(row)
            cursor.execute(sql_c, row)
            
        first = False
    
conn.commit()
cursor.close()
conn.close()

# 2 Dimension Table Genre

## 2.1 Extract and transform data

In [15]:
## Connect to 'daydayup_db' to extract data
conn = make_connection(config_file = 'config_sjsu_daydayup_db2.ini')
cursor = conn.cursor()

In [16]:
## Extract the data for Track factable
sql = ( """
        SELECT * from Genre
        """
      )

df_genre = pd.read_sql(sql, conn)
df_genre

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll
5,6,Blues
6,7,Latin
7,8,Reggae
8,9,Pop
9,10,Soundtrack


In [17]:
df_genre.shape

(25, 2)

In [18]:
df_genre['GenreKey'] = [i for i in range(1, df_genre.shape[0]+1)]
df_genre['GenreName'] = df_genre.Name

## Suppose we can divide the genre by ID into 2 catogories: "Classical" and "Modern"
df_genre['GenreType'] = "Modern"
df_genre.GenreType[df_genre.GenreId < 20] = "Classic"

In [19]:
df_genre.GenreType.unique()

array(['Classic', 'Modern'], dtype=object)

In [20]:
df_genre_wh2 = df_genre[['GenreKey', 'GenreId', 'GenreName', 'GenreType']]
df_genre_wh2

,GenreKey,GenreId,GenreName,GenreType
0,1,1,Rock,Classic
1,2,2,Jazz,Classic
2,3,3,Metal,Classic
3,4,4,Alternative & Punk,Classic
4,5,5,Rock And Roll,Classic
5,6,6,Blues,Classic
6,7,7,Latin,Classic
7,8,8,Reggae,Classic
8,9,9,Pop,Classic
9,10,10,Soundtrack,Classic


In [21]:
## Missing values
df_genre_wh2.isnull().sum()

GenreKey     0
GenreId      0
GenreName    0
GenreType    0
dtype: int64

In [22]:
## Export to csv
df_genre_wh2.to_csv('wh2_genre.csv', index=False)
cursor.close()
conn.close()

## 2.2 Create and populate the dimension table Genre (wh2)

In [23]:
## Connect to 'daydayup_wh' to create dimension table
conn = make_connection(config_file = 'config_sjsu_daydayup_wh2.ini')
cursor = conn.cursor()

In [24]:
## Drop table if already exist
## cursor.execute('DROP TABLE IF EXISTS Sales_Fact_Table')
## cursor.execute('DROP TABLE IF EXISTS Cancels_Fact_Table')
cursor.execute('DROP TABLE IF EXISTS Track')
cursor.execute('DROP TABLE IF EXISTS Genre')

## create the dimension table PRODUCT
sql = ('''
CREATE TABLE `Genre`
(
    `GenreKey` INT NOT NULL AUTO_INCREMENT,
    `GenreId` INT NOT NULL,
    `GenreName` NVARCHAR(200) NOT NULL,
    `GenreType` NVARCHAR(20) NOT NULL,
     PRIMARY KEY  (`GenreKey`)
)

        ''')

cursor.execute(sql)

In [25]:
## Populate the dimension table Track
sql = ( "INSERT INTO Genre \n"
        + "VALUES (%s, %s, %s, %s)"
        )


first = True ## Skip the first row

with open('wh2_genre.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            #transform(row)
            cursor.execute(sql, row)
            
        first = False
    
conn.commit()
cursor.close()
conn.close()

# 3 Dimension Table Location

## 3.1 Extract and transform data

In [26]:
## Connect to 'daydayup_db' to extract data
conn = make_connection(config_file = 'config_sjsu_daydayup_db2.ini')

## Txtract the data
## The location data of billing address and customer is identical, so we extract the
## Location info from Customer and use CustomerID as a connecting key
sql = ( """
        SELECT * from Customer
        """
      )
df_location = pd.read_sql(sql, conn)

df_location

,CustomerId,FirstName,LastName,Address,City,State,Country,PostalCode,Phone,Email,SupportRepId
0,1,Luís,Gonçalves,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,luisg@embraer.com.br,3
1,2,Leonie,Köhler,Theodor-Heuss-Straße 34,Stuttgart,Unknown,Germany,70174,+49 0711 2842222,leonekohler@surfeu.de,5
2,3,François,Tremblay,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,Ullevålsveien 14,Oslo,Unknown,Norway,0171,+47 22 44 22 22,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,Klanova 9/506,Prague,Unknown,Czech Republic,14700,+420 2 4172 5555,frantisekw@jetbrains.com,4
5,6,Helena,Holý,Rilská 3174/6,Prague,Unknown,Czech Republic,14300,+420 2 4177 0449,hholy@gmail.com,5
6,7,Astrid,Gruber,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,Unknown,Austria,1010,+43 01 5134505,astrid.gruber@apple.at,5
7,8,Daan,Peeters,Grétrystraat 63,Brussels,Unknown,Belgium,1000,+32 02 219 03 03,daan_peeters@apple.be,4
8,9,Kara,Nielsen,Sønder Boulevard 51,Copenhagen,Unknown,Denmark,1720,+453 3331 9991,kara.nielsen@jubii.dk,4
9,10,Eduardo,Martins,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,eduardo@woodstock.com.br,4


In [27]:
df_location['LocationKey'] = [i for i in range(1, df_location.shape[0]+1)]
df_location.head()

,CustomerId,FirstName,LastName,Address,City,State,Country,PostalCode,Phone,Email,SupportRepId,LocationKey
0,1,Luís,Gonçalves,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,luisg@embraer.com.br,3,1
1,2,Leonie,Köhler,Theodor-Heuss-Straße 34,Stuttgart,Unknown,Germany,70174,+49 0711 2842222,leonekohler@surfeu.de,5,2
2,3,François,Tremblay,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,ftremblay@gmail.com,3,3
3,4,Bjørn,Hansen,Ullevålsveien 14,Oslo,Unknown,Norway,0171,+47 22 44 22 22,bjorn.hansen@yahoo.no,4,4
4,5,František,Wichterlová,Klanova 9/506,Prague,Unknown,Czech Republic,14700,+420 2 4172 5555,frantisekw@jetbrains.com,4,5


In [28]:
## Select needed columns
df_location_wh2 = df_location[['LocationKey', 'CustomerId', 'Address', 'City', 'State', 'Country', 'PostalCode']]
df_location_wh2.head()

,LocationKey,CustomerId,Address,City,State,Country,PostalCode
0,1,1,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000
1,2,2,Theodor-Heuss-Straße 34,Stuttgart,Unknown,Germany,70174
2,3,3,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7
3,4,4,Ullevålsveien 14,Oslo,Unknown,Norway,0171
4,5,5,Klanova 9/506,Prague,Unknown,Czech Republic,14700


In [29]:
## Export to csv
df_location_wh2.to_csv('wh2_location.csv', index=False)
## cursor.close()
conn.close()

## 3.2 Create and populate the dimension table location (wh2)

In [30]:
## Connect to 'daydayup_wh' to create dimension table
conn = make_connection(config_file = 'config_sjsu_daydayup_wh2.ini')
cursor = conn.cursor()

In [31]:
## Drop table if already exist
## cursor.execute('DROP TABLE IF EXISTS Sales_Fact_Table')
## cursor.execute('DROP TABLE IF EXISTS Cancels_Fact_Table')
cursor.execute('DROP TABLE IF EXISTS Location')

## Create the dimension table Album
sql = ('''
CREATE TABLE `Location`
(
    `LocationKey` INT NOT NULL AUTO_INCREMENT,
    `CustomerId` INT NOT NULL,
    `Address` NVARCHAR(200) NOT NULL,
    `City` NVARCHAR(50) NOT NULL,
    `State` NVARCHAR(50) NOT NULL,
    `Country` NVARCHAR(50) NOT NULL,
    `PostalCode` NVARCHAR(50) NOT NULL,
    PRIMARY KEY  (`LocationKey`)
)
        ''')

cursor.execute(sql)

In [32]:
## Populate the dimension table STORE
sql = ( "INSERT INTO Location \n"
        + "VALUES (%s, %s, %s, %s, %s,  %s, %s)"
        )


first = True ## Skip the first row

with open('wh2_Location.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            #transform(row)
            cursor.execute(sql, row)
            
        first = False
    
conn.commit()
cursor.close()
conn.close()

# 4 Dimension Table Timedetail

## 4.1 Extract and transform data

In [33]:
## Connect to database
conn = make_connection(config_file = 'config_sjsu_daydayup_db2.ini')

## Extract the data 
sql = ( """
        SELECT DISTINCT InvoiceDate from Invoice
        """
      )
df_time = pd.read_sql(sql, conn)
df_time

,InvoiceDate
0,2009-01-01
1,2009-01-02
2,2009-01-03
3,2009-01-06
4,2009-01-11
...,...
349,2013-12-05
350,2013-12-06
351,2013-12-09
352,2013-12-14


In [34]:
df_time['TimeKey'] = [i for i in range(1, df_time.shape[0]+1)]
df_time

,InvoiceDate,TimeKey
0,2009-01-01,1
1,2009-01-02,2
2,2009-01-03,3
3,2009-01-06,4
4,2009-01-11,5
...,...,...
349,2013-12-05,350
350,2013-12-06,351
351,2013-12-09,352
352,2013-12-14,353


In [35]:
## Transform the data
df_time['PurchaseDate'] = df_time['InvoiceDate']
df_time['DateOfWeek'] = pd.DatetimeIndex(df_time['InvoiceDate']).dayofweek+1 ## Monday=1, Sunday=7
df_time['Month'] = pd.DatetimeIndex(df_time['InvoiceDate']).month
df_time['Quarter'] = pd.DatetimeIndex(df_time['InvoiceDate']).quarter
df_time['Year'] = pd.DatetimeIndex(df_time['InvoiceDate']).year

## Display the transformed data
df_time

,InvoiceDate,TimeKey,PurchaseDate,DateOfWeek,Month,Quarter,Year
0,2009-01-01,1,2009-01-01,4,1,1,2009
1,2009-01-02,2,2009-01-02,5,1,1,2009
2,2009-01-03,3,2009-01-03,6,1,1,2009
3,2009-01-06,4,2009-01-06,2,1,1,2009
4,2009-01-11,5,2009-01-11,7,1,1,2009
...,...,...,...,...,...,...,...
349,2013-12-05,350,2013-12-05,4,12,4,2013
350,2013-12-06,351,2013-12-06,5,12,4,2013
351,2013-12-09,352,2013-12-09,1,12,4,2013
352,2013-12-14,353,2013-12-14,6,12,4,2013


In [36]:
## Select needed columns
df_time_wh2 = df_time[['TimeKey', 'PurchaseDate', 'DateOfWeek', 'Month', 'Quarter', 'Year']]
df_time_wh2

,TimeKey,PurchaseDate,DateOfWeek,Month,Quarter,Year
0,1,2009-01-01,4,1,1,2009
1,2,2009-01-02,5,1,1,2009
2,3,2009-01-03,6,1,1,2009
3,4,2009-01-06,2,1,1,2009
4,5,2009-01-11,7,1,1,2009
...,...,...,...,...,...,...
349,350,2013-12-05,4,12,4,2013
350,351,2013-12-06,5,12,4,2013
351,352,2013-12-09,1,12,4,2013
352,353,2013-12-14,6,12,4,2013


In [37]:
## Export to csv
df_time_wh2.to_csv('wh2_time.csv', index=False)
conn.close()

In [38]:
df_time_wh2.isnull().sum()

TimeKey         0
PurchaseDate    0
DateOfWeek      0
Month           0
Quarter         0
Year            0
dtype: int64

In [39]:
cursor.close()
conn.close()

## 4.2 Create and populate the dimension table Timedetail

In [40]:
## Connect to 'daydayup_wh' to create dimension table
conn = make_connection(config_file = 'config_sjsu_daydayup_wh2.ini')
cursor = conn.cursor()

In [41]:
## Drop table if already exist
cursor.execute('DROP TABLE IF EXISTS Sales_Fact_Table')
cursor.execute('DROP TABLE IF EXISTS Preference_Fact_Table')
cursor.execute('DROP TABLE IF EXISTS Timedetail')

## Create the dimension table TIMEDETAIL
sql = ('''
        CREATE TABLE Timedetail
        ( TimeKey INT NOT NULL AUTO_INCREMENT,
          PurchaseDate DATE NOT NULL,
          DayOfWeek INT NOT NULL,
          Month INT NOT NULL,
          Quarter INT NOT NULL,
          Year INT NOT NULL,
          PRIMARY KEY (TimeKey) )
        ''')

cursor.execute(sql)

In [42]:
## Populate the dimension table Timedetail
sql_t = ( "INSERT INTO Timedetail \n"
        + "VALUES (%s, %s, %s, %s, %s, %s)"
        )


first = True ## Skip the first row

with open('wh2_time.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            #transform(row)
            cursor.execute(sql_t, row)
            
        first = False
    
conn.commit()
cursor.close()
conn.close()

# 5 Diamensional Table: Track

## 5.1 Extract and Transform

In [43]:
## Connect to database
conn = make_connection(config_file = 'config_sjsu_daydayup_db2.ini')
cursor = conn.cursor()

In [44]:
sql = ( """
        SELECT Track.TrackId, Track.Name TrackName, Track.Composer TrackComposer, 
        Track.Milliseconds, Track.Bytes, Track.UnitPrice,
        
        
        
        MediaType.Name MediaTypeName,
        Album.AlbumId, Album.Title AlbumTitle, Album.ArtistId, 
        Artist.Name ArtistName
        
        from Track, Album, Artist, MediaType 
        
        
        where Track.`MediaTypeId` = MediaType.`MediaTypeId`
        and Track.AlbumId=Album.AlbumId
        and Album.ArtistId=Artist.ArtistId
        """
      )

df_track = pd.read_sql(sql, conn)
df_track

,TrackId,TrackName,TrackComposer,Milliseconds,Bytes,UnitPrice,MediaTypeName,AlbumId,AlbumTitle,ArtistId,ArtistName
0,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC
1,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC
2,7,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC
3,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC
4,9,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC
...,...,...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) I Pini Della Vi...,unknown,286741,4718950,0.99,Protected AAC audio file,343,Respighi:Pines of Rome,226,Eugene Ormandy
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Franz Schubert,139200,2283131,0.99,Protected AAC audio file,344,Schubert: The Late String Quartets & String Qu...,272,Emerson String Quartet
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",Claudio Monteverdi,66639,1189062,0.99,Protected AAC audio file,345,Monteverdi: L'Orfeo,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Wolfgang Amadeus Mozart,221331,3665114,0.99,Protected AAC audio file,346,Mozart: Chamber Music,274,Nash Ensemble


In [45]:
df_track['TrackKey'] = [i for i in range(1, df_track.shape[0]+1)]
df_track

,TrackId,TrackName,TrackComposer,Milliseconds,Bytes,UnitPrice,MediaTypeName,AlbumId,AlbumTitle,ArtistId,ArtistName,TrackKey
0,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC,1
1,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC,2
2,7,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC,3
3,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC,4
4,9,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC,5
...,...,...,...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) I Pini Della Vi...,unknown,286741,4718950,0.99,Protected AAC audio file,343,Respighi:Pines of Rome,226,Eugene Ormandy,3499
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Franz Schubert,139200,2283131,0.99,Protected AAC audio file,344,Schubert: The Late String Quartets & String Qu...,272,Emerson String Quartet,3500
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",Claudio Monteverdi,66639,1189062,0.99,Protected AAC audio file,345,Monteverdi: L'Orfeo,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",3501
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Wolfgang Amadeus Mozart,221331,3665114,0.99,Protected AAC audio file,346,Mozart: Chamber Music,274,Nash Ensemble,3502


In [46]:
df_track_wh2 = df_track[['TrackKey'] + ['TrackId', 'TrackName', 'TrackComposer', 'Milliseconds', 'Bytes',
       'UnitPrice', 'MediaTypeName', 'AlbumId', 'AlbumTitle', 'ArtistId',
       'ArtistName']]

In [47]:
df_track_wh2.head()

,TrackKey,TrackId,TrackName,TrackComposer,Milliseconds,Bytes,UnitPrice,MediaTypeName,AlbumId,AlbumTitle,ArtistId,ArtistName
0,1,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC
1,2,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC
2,3,7,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC
3,4,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC
4,5,9,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,MPEG audio file,1,For Those About To Rock We Salute You,1,AC/DC


In [48]:
df_track_wh2.isnull().sum()

TrackKey         0
TrackId          0
TrackName        0
TrackComposer    0
Milliseconds     0
Bytes            0
UnitPrice        0
MediaTypeName    0
AlbumId          0
AlbumTitle       0
ArtistId         0
ArtistName       0
dtype: int64

In [49]:
df_track_wh2.to_csv("wh2_track.csv", index=False)
cursor.close()
conn.close()

## 5.2 Populate the dimensional table

In [50]:
## Connect to 'daydayup_wh' to create dimension table
conn = make_connection(config_file = 'config_sjsu_daydayup_wh2.ini')
cursor = conn.cursor()

In [51]:
## Drop table if already exist
cursor.execute('DROP TABLE IF EXISTS Sales_Fact_Table')
cursor.execute('DROP TABLE IF EXISTS Preference_Fact_Table')
cursor.execute('DROP TABLE IF EXISTS Track')

## Create the dimension table TIMEDETAIL
sql = ('''
        CREATE TABLE Track
        ( TrackKey INT NOT NULL AUTO_INCREMENT,
          TrackId INT NOT NULL,
          TrackName NVARCHAR(200) NOT NULL,
          TrackComposer NVARCHAR(220) NOT NULL, 
          Milliseconds INT NOT NULL, 
          Bytes INT NOT NULL, 
          UnitPrice NUMERIC(10,2) NOT NULL, 
          MediaTypeName NVARCHAR(50) NOT NULL, 
          AlbumId INT NOT NULL, 
          AlbumTitle NVARCHAR(220) NOT NULL , 
          ArtistId INT NOT NULL,
          ArtistName NVARCHAR(100) NOT NULL,
          PRIMARY KEY  (TrackKey) 
          )
        ''')

cursor.execute(sql)

In [52]:
## Populate the dimension table Track
sql = ( "INSERT INTO Track \n"
        + "VALUES (%s, %s, %s, %s, %s,  %s, %s, %s, %s, %s,  %s, %s)"
        )


first = True ## Skip the first row

with open('wh2_track.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            #transform(row)
            cursor.execute(sql, row)
            
        first = False
    
conn.commit()
cursor.close()
conn.close()

# 6 Fact Table: Preference 

## 6.1 Extract and transform data

In [53]:
## Connect to 'daydayup_db' to extract data
conn = make_connection(config_file = 'config_sjsu_daydayup_db2.ini')

## Extract the data
## We use the name "PurchaseDate" for "AddingDate" to merge for the "TimeKey"
sql = ( """
        SELECT Customer.CustomerId, Track.TrackId, PlaylistTrack.PlaylistId, Playlist.Name PlaylistName, PlaylistTrack.AddingDate as PurchaseDate, Genre.GenreId
        from Customer, PlaylistTrack, Track, Playlist, Genre, MediaType, Album
        where PlaylistTrack.TrackId = Track.TrackId
        and PlaylistTrack.PlaylistId = Playlist.PlaylistId
        and Customer.CustomerId = Playlist.CustomerId

        
        and Track.TrackId=PlaylistTrack.TrackId
        and Track.GenreId=Genre.GenreId
        and Track.MediaTypeId = MediaType.MediaTypeId
        and Track.AlbumId = Album.AlbumId
        """
      )
df_pref_all = pd.read_sql(sql, conn)

df_pref_all

,CustomerId,TrackId,PlaylistId,PlaylistName,PurchaseDate,GenreId
0,9,3353,1,Music,2013-03-18,1
1,8,3353,8,Music,2011-07-22,1
2,9,3355,1,Music,2010-04-16,1
3,8,3355,8,Music,2009-12-08,1
4,9,2,1,Music,2013-02-02,1
...,...,...,...,...,...,...
8710,9,3451,1,Music,2011-07-20,25
8711,49,3451,5,90’s Music,2010-05-13,25
8712,8,3451,8,Music,2010-05-22,25
8713,31,3451,12,Classical,2012-09-26,25


In [54]:
## Merge Customer Key
df_customer_wh2.head()

,CustomerKey,CustomerId,FirstName,LastName,Gender,Age,Phone,Email,SupportRepId
0,1,1,Luís,Gonçalves,F,18,+55 (12) 3923-5555,luisg@embraer.com.br,3
1,2,2,Leonie,Köhler,M,35,+49 0711 2842222,leonekohler@surfeu.de,5
2,3,3,François,Tremblay,F,30,+1 (514) 721-4711,ftremblay@gmail.com,3
3,4,4,Bjørn,Hansen,F,55,+47 22 44 22 22,bjorn.hansen@yahoo.no,4
4,5,5,František,Wichterlová,M,32,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [55]:
df_temp = df_customer_wh2[['CustomerId', 'CustomerKey']]
df_temp.head()

,CustomerId,CustomerKey
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [56]:
## Merge the CustomerKey to the fact table
## result = pd.merge(left, right, how="left", on=["key1", "key2"])
df_pref_temp1 = pd.merge(df_pref_all, df_temp, how="left", on="CustomerId")
df_pref_temp1

,CustomerId,TrackId,PlaylistId,PlaylistName,PurchaseDate,GenreId,CustomerKey
0,9,3353,1,Music,2013-03-18,1,9
1,8,3353,8,Music,2011-07-22,1,8
2,9,3355,1,Music,2010-04-16,1,9
3,8,3355,8,Music,2009-12-08,1,8
4,9,2,1,Music,2013-02-02,1,9
...,...,...,...,...,...,...,...
8710,9,3451,1,Music,2011-07-20,25,9
8711,49,3451,5,90’s Music,2010-05-13,25,49
8712,8,3451,8,Music,2010-05-22,25,8
8713,31,3451,12,Classical,2012-09-26,25,31


In [57]:
## Merge the TrackKey with the fact table
df_genre_wh2.head()

,GenreKey,GenreId,GenreName,GenreType
0,1,1,Rock,Classic
1,2,2,Jazz,Classic
2,3,3,Metal,Classic
3,4,4,Alternative & Punk,Classic
4,5,5,Rock And Roll,Classic


In [58]:
df_temp = df_genre_wh2[['GenreId','GenreKey']]
df_temp.head()

,GenreId,GenreKey
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [59]:
## Merge
df_pref_temp2 = pd.merge(df_pref_temp1, df_temp, how="left", on="GenreId")
df_pref_temp2

,CustomerId,TrackId,PlaylistId,PlaylistName,PurchaseDate,GenreId,CustomerKey,GenreKey
0,9,3353,1,Music,2013-03-18,1,9,1
1,8,3353,8,Music,2011-07-22,1,8,1
2,9,3355,1,Music,2010-04-16,1,9,1
3,8,3355,8,Music,2009-12-08,1,8,1
4,9,2,1,Music,2013-02-02,1,9,1
...,...,...,...,...,...,...,...,...
8710,9,3451,1,Music,2011-07-20,25,9,25
8711,49,3451,5,90’s Music,2010-05-13,25,49,25
8712,8,3451,8,Music,2010-05-22,25,8,25
8713,31,3451,12,Classical,2012-09-26,25,31,25


In [60]:
## Merge Location
df_location_wh2

,LocationKey,CustomerId,Address,City,State,Country,PostalCode
0,1,1,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000
1,2,2,Theodor-Heuss-Straße 34,Stuttgart,Unknown,Germany,70174
2,3,3,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7
3,4,4,Ullevålsveien 14,Oslo,Unknown,Norway,0171
4,5,5,Klanova 9/506,Prague,Unknown,Czech Republic,14700
5,6,6,Rilská 3174/6,Prague,Unknown,Czech Republic,14300
6,7,7,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,Unknown,Austria,1010
7,8,8,Grétrystraat 63,Brussels,Unknown,Belgium,1000
8,9,9,Sønder Boulevard 51,Copenhagen,Unknown,Denmark,1720
9,10,10,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010


In [61]:
df_pref_temp3 = df_pref_temp2
df_pref_temp3['LocationKey'] = df_pref_temp3.CustomerId
df_pref_temp3

,CustomerId,TrackId,PlaylistId,PlaylistName,PurchaseDate,GenreId,CustomerKey,GenreKey,LocationKey
0,9,3353,1,Music,2013-03-18,1,9,1,9
1,8,3353,8,Music,2011-07-22,1,8,1,8
2,9,3355,1,Music,2010-04-16,1,9,1,9
3,8,3355,8,Music,2009-12-08,1,8,1,8
4,9,2,1,Music,2013-02-02,1,9,1,9
...,...,...,...,...,...,...,...,...,...
8710,9,3451,1,Music,2011-07-20,25,9,25,9
8711,49,3451,5,90’s Music,2010-05-13,25,49,25,49
8712,8,3451,8,Music,2010-05-22,25,8,25,8
8713,31,3451,12,Classical,2012-09-26,25,31,25,31


In [62]:
## Merge TimeKey
df_time_wh2.head()

,TimeKey,PurchaseDate,DateOfWeek,Month,Quarter,Year
0,1,2009-01-01,4,1,1,2009
1,2,2009-01-02,5,1,1,2009
2,3,2009-01-03,6,1,1,2009
3,4,2009-01-06,2,1,1,2009
4,5,2009-01-11,7,1,1,2009


In [63]:
df_temp = df_time_wh2[['TimeKey', 'PurchaseDate']]
df_temp.head()

,TimeKey,PurchaseDate
0,1,2009-01-01
1,2,2009-01-02
2,3,2009-01-03
3,4,2009-01-06
4,5,2009-01-11


In [64]:
df_temp.PurchaseDate = pd.to_datetime(df_temp.PurchaseDate)

In [65]:
df_pref_temp3.PurchaseDate = pd.to_datetime(df_pref_temp3.PurchaseDate)

In [66]:
## merge
df_pref_temp4 = pd.merge(df_pref_temp3, df_temp, how="left", on="PurchaseDate")
df_pref_temp4.head()

,CustomerId,TrackId,PlaylistId,PlaylistName,PurchaseDate,GenreId,CustomerKey,GenreKey,LocationKey,TimeKey
0,9,3353,1,Music,2013-03-18,1,9,1,9,300
1,8,3353,8,Music,2011-07-22,1,8,1,8,183
2,9,3355,1,Music,2010-04-16,1,9,1,9,94
3,8,3355,8,Music,2009-12-08,1,8,1,8,67
4,9,2,1,Music,2013-02-02,1,9,1,9,292


In [67]:
## Merge TrackKey
df_pref_temp4['TrackKey'] = df_pref_temp4.TrackId

In [68]:
df_pref_temp4.isnull().sum()

CustomerId      0
TrackId         0
PlaylistId      0
PlaylistName    0
PurchaseDate    0
GenreId         0
CustomerKey     0
GenreKey        0
LocationKey     0
TimeKey         0
TrackKey        0
dtype: int64

In [69]:
df_pref_wh2 = df_pref_temp4[['CustomerKey', 'TrackKey', 'PlaylistId', 'PlaylistName', 'LocationKey', 'TimeKey', 'GenreKey']]
df_pref_wh2.head()

,CustomerKey,TrackKey,PlaylistId,PlaylistName,LocationKey,TimeKey,GenreKey
0,9,3353,1,Music,9,300,1
1,8,3353,8,Music,8,183,1
2,9,3355,1,Music,9,94,1
3,8,3355,8,Music,8,67,1
4,9,2,1,Music,9,292,1


In [70]:
df_pref_wh2.loc[df_pref_wh2.duplicated(subset=['TrackKey', 'PlaylistId']) == True]

,CustomerKey,TrackKey,PlaylistId,PlaylistName,LocationKey,TimeKey,GenreKey


In [71]:
## Drop duplicates
## df_pref_wh2.drop_duplicates(subset=['TrackKey', 'PlaylistId'], keep='first', inplace=True)
## df_pref_wh2.loc[df_pref_wh2.duplicated(subset=['TrackKey', 'PlaylistId']) == True]

In [72]:
df_pref_wh2.to_csv("wh2_pref.csv", index=False)
cursor.close()
conn.close()

## 6.2 Create and populate the preference fact table

In [73]:
## Connect to 'daydayup_wh' to create fact table
conn = make_connection(config_file = 'config_sjsu_daydayup_wh2.ini')
cursor = conn.cursor()

In [74]:
## Drop table if already exist
cursor.execute('DROP TABLE IF EXISTS Preference_Fact_Table')

## Create the dimension table Cancels_Fact_Table
sql = ('''
        CREATE TABLE Preference_Fact_Table
        ( 
          CustomerKey INT NOT NULL,
          TrackKey INT NOT NULL,
          PlaylistId INT NOT NULL,
          PlaylistName VARCHAR(120) not null,
          LocationKey INT NOT NULL,
          TimeKey INT NOT NULL,
          GenreKey INT NOT NULL, 
          
          FOREIGN KEY (TrackKey) REFERENCES Track(TrackKey),
          FOREIGN KEY (CustomerKey) REFERENCES Customer(CustomerKey),
          FOREIGN KEY (TimeKey) REFERENCES Timedetail(TimeKey),
          FOREIGN KEY (LocationKey) REFERENCES Location(LocationKey),
          PRIMARY KEY (TrackKey, PlaylistId)
          )
        ''')

cursor.execute(sql)

In [75]:
## Populate the Cancels_Fact_Table
sql_can = ( "INSERT INTO Preference_Fact_Table \n"
        + "VALUES (%s, %s, %s, %s, %s,  %s, %s)"
        )


first = True ## Skip the first row

with open('wh2_pref.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            #transform(row)
            cursor.execute(sql_can, row)
            
        first = False
    
conn.commit()
cursor.close()
conn.close()

# 7 Fact Table: Sales

## 7.1 Extract and transform data

In [76]:
## Connect to 'daydayup_db' to extract data
conn = make_connection(config_file = 'config_sjsu_daydayup_db2.ini')

## Extract the data
sql = ( """
        SELECT Invoice.InvoiceId, InvoiceLine.InvoiceLineId, Customer.CustomerId, Track.TrackId, Invoice.InvoiceDate as PurchaseDate, InvoiceLine.UnitPrice
        from InvoiceLine, Customer, Track, Invoice
        where Customer.CustomerId = Invoice.CustomerId
        and Invoice.invoiceId = InvoiceLine.InvoiceId
        and InvoiceLine.TrackId = Track.TrackId
        """
      )
df_sales_all = pd.read_sql(sql, conn)

df_sales_all.sort_values(by='PurchaseDate')

,InvoiceId,InvoiceLineId,CustomerId,TrackId,PurchaseDate,UnitPrice
0,1,1,2,2,2009-01-01,0.99
1,1,2,2,4,2009-01-01,0.99
2,2,3,4,6,2009-01-02,0.99
3,2,4,4,8,2009-01-02,0.99
4,2,5,4,10,2009-01-02,0.99
...,...,...,...,...,...,...
745,138,746,37,1022,2010-08-23,0.99
744,138,745,37,1013,2010-08-23,0.99
743,138,744,37,1004,2010-08-23,0.99
749,138,750,37,1058,2010-08-23,0.99


In [77]:
## Merge CustomerKey
df_temp = df_customer_wh2[['CustomerId', 'CustomerKey']]
df_temp.head()

,CustomerId,CustomerKey
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [78]:
df_sales_temp1 = pd.merge(df_sales_all, df_temp, how="left", on="CustomerId")
df_sales_temp1.sort_values(by='PurchaseDate')

,InvoiceId,InvoiceLineId,CustomerId,TrackId,PurchaseDate,UnitPrice,CustomerKey
0,1,1,2,2,2009-01-01,0.99,2
1,1,2,2,4,2009-01-01,0.99,2
2,2,3,4,6,2009-01-02,0.99,4
3,2,4,4,8,2009-01-02,0.99,4
4,2,5,4,10,2009-01-02,0.99,4
...,...,...,...,...,...,...,...
745,138,746,37,1022,2010-08-23,0.99,37
744,138,745,37,1013,2010-08-23,0.99,37
743,138,744,37,1004,2010-08-23,0.99,37
749,138,750,37,1058,2010-08-23,0.99,37


In [79]:
## Merge TrackKey
df_temp = df_track_wh2[['TrackId','TrackKey']]
df_temp.head()

,TrackId,TrackKey
0,1,1
1,6,2
2,7,3
3,8,4
4,9,5


In [80]:
df_sales_temp2 = pd.merge(df_sales_temp1, df_temp, how="left", on="TrackId")
df_sales_temp2

,InvoiceId,InvoiceLineId,CustomerId,TrackId,PurchaseDate,UnitPrice,CustomerKey,TrackKey
0,1,1,2,2,2009-01-01,0.99,2,11
1,1,2,2,4,2009-01-01,0.99,2,13
2,2,3,4,6,2009-01-02,0.99,4,2
3,2,4,4,8,2009-01-02,0.99,4,4
4,2,5,4,10,2009-01-02,0.99,4,6
...,...,...,...,...,...,...,...,...
753,138,754,37,1094,2010-08-23,0.99,37,1110
754,138,755,37,1103,2010-08-23,0.99,37,1119
755,138,756,37,1112,2010-08-23,0.99,37,930
756,138,757,37,1121,2010-08-23,0.99,37,1121


In [81]:
## Merge time detail
df_time_wh2

,TimeKey,PurchaseDate,DateOfWeek,Month,Quarter,Year
0,1,2009-01-01,4,1,1,2009
1,2,2009-01-02,5,1,1,2009
2,3,2009-01-03,6,1,1,2009
3,4,2009-01-06,2,1,1,2009
4,5,2009-01-11,7,1,1,2009
...,...,...,...,...,...,...
349,350,2013-12-05,4,12,4,2013
350,351,2013-12-06,5,12,4,2013
351,352,2013-12-09,1,12,4,2013
352,353,2013-12-14,6,12,4,2013


In [82]:
df_temp = df_time_wh2[['TimeKey', 'PurchaseDate']]
df_temp

,TimeKey,PurchaseDate
0,1,2009-01-01
1,2,2009-01-02
2,3,2009-01-03
3,4,2009-01-06
4,5,2009-01-11
...,...,...
349,350,2013-12-05
350,351,2013-12-06
351,352,2013-12-09
352,353,2013-12-14


In [83]:
## Merge
df_sales_temp3 = pd.merge(df_sales_temp2, df_temp, how="left", on="PurchaseDate")
df_sales_temp3.sort_values(by=['PurchaseDate'])

,InvoiceId,InvoiceLineId,CustomerId,TrackId,PurchaseDate,UnitPrice,CustomerKey,TrackKey,TimeKey
0,1,1,2,2,2009-01-01,0.99,2,11,1
1,1,2,2,4,2009-01-01,0.99,2,13,1
2,2,3,4,6,2009-01-02,0.99,4,2,2
3,2,4,4,8,2009-01-02,0.99,4,4,2
4,2,5,4,10,2009-01-02,0.99,4,6,2
...,...,...,...,...,...,...,...,...,...
745,138,746,37,1022,2010-08-23,0.99,37,1038,119
744,138,745,37,1013,2010-08-23,0.99,37,1029,119
743,138,744,37,1004,2010-08-23,0.99,37,1020,119
749,138,750,37,1058,2010-08-23,0.99,37,1074,119


In [84]:
## Merge LocationKey
df_sales_temp3['LocationKey'] = df_sales_temp3.CustomerKey

In [85]:
df_sales_temp3.isnull().sum()

InvoiceId        0
InvoiceLineId    0
CustomerId       0
TrackId          0
PurchaseDate     0
UnitPrice        0
CustomerKey      0
TrackKey         0
TimeKey          0
LocationKey      0
dtype: int64

In [86]:
df_sales_wh2 = df_sales_temp3[['CustomerKey', 'LocationKey', 'TimeKey', 'InvoiceLineId', 'InvoiceId', 'TrackKey', 'UnitPrice']]
df_sales_wh2.to_csv("df_sales_wh2.csv", index=False)

In [87]:
## Export to csv
df_sales_wh2.to_csv('wh2_sales.csv', index = False)
cursor.close()
conn.close()

In [88]:
df_sales_wh2

,CustomerKey,LocationKey,TimeKey,InvoiceLineId,InvoiceId,TrackKey,UnitPrice
0,2,2,1,1,1,11,0.99
1,2,2,1,2,1,13,0.99
2,4,4,2,3,2,2,0.99
3,4,4,2,4,2,4,0.99
4,4,4,2,5,2,6,0.99
...,...,...,...,...,...,...,...
753,37,37,119,754,138,1110,0.99
754,37,37,119,755,138,1119,0.99
755,37,37,119,756,138,930,0.99
756,37,37,119,757,138,1121,0.99


## 7.2 Create and populate Sales_Fact_Table

In [89]:
## Connect to 'daydayup_wh' to create fact table
conn = make_connection(config_file = 'config_sjsu_daydayup_wh2.ini')
cursor = conn.cursor()
conn

In [90]:
## Drop table if already exist
cursor.execute('DROP TABLE IF EXISTS Sales_Fact_Table')

In [91]:
## Create the dimension table Sales_Fact_Table
sql = ('''
        CREATE TABLE Sales_Fact_Table
        (
          
          CustomerKey INT NOT NULL,
          LocationKey INT NOT NULL,
          TimeKey INT NOT NULL,
          InvoiceLineId INT NOT NULL,
          InvoiceId INT NOT NULL,
          TrackKey INT NOT NULL,
          UnitPrice FLOAT NOT NULL,
          
          
          FOREIGN KEY (CustomerKey) REFERENCES Customer(CustomerKey),
          FOREIGN KEY (TrackKey) REFERENCES Track(TrackKey),
          FOREIGN KEY (TimeKey) REFERENCES Timedetail(TimeKey),
          FOREIGN KEY (LocationKey) REFERENCES Location(LocationKey),
          
          PRIMARY KEY (InvoiceLineId)
        )
        ''')

cursor.execute(sql)

Cannot add or update a child row: a foreign key constraint fails (`daydayup_wh2`.`Sales_Fact_Table`, CONSTRAINT `Sales_Fact_Table_ibfk_3` FOREIGN KEY (`TimeKey`) REFERENCES `Timedetail` (`TimeKey`))

In [92]:
df_sales_wh2.TimeKey.value_counts()

47     14
101    14
59     14
77     14
23     14
       ..
72      1
78      1
84      1
90      1
120     1
Name: TimeKey, Length: 120, dtype: int64

In [93]:
df_sales_wh2.TimeKey.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120])

In [94]:
df_sales_wh2.shape

(758, 7)

In [95]:
## Populate the Table
sql = ( "INSERT INTO Sales_Fact_Table \n"
        + "VALUES (%s, %s, %s, %s, %s,   %s, %s)"
        )


first = True ## Skip the first row

with open('wh2_sales.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            #transform(row)
            cursor.execute(sql, row)
            
        first = False
    
conn.commit()
cursor.close()
conn.close()